# AuxTel spectrum seeing
Craig Lage 21-Jan-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import pickle as pkl
from astropy.time import Time, TimeDelta
import pandas as pd
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.summit.utils.plotting import plot
import lsst.afw.cameraGeom.utils as camGeomUtils
import lsst.afw.display as afwDisplay
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from scipy.optimize import minimize

## Define a simple ISR

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

In [ ]:
def FOM(params, args):
    fom = 0.0
    [xs, fluxes] = args
    [bg, mag, mean, sigma] = params
    for i in range(len(xs)):
        model = bg + mag * np.exp(-(xs[i] - mean)**2 / (2 * sigma**2))
        fom += np.square(model - fluxes[i])
    return fom


## The cell below took way too long, so I converted it into a Python script for running.

In [ ]:
y = 2000
x1 = 1600
x2 = 1900

butler = Butler('/repo/embargo_old', collections="LATISS/raw/all")
startDay = 20240708
endDay = 20250120
dayObs = startDay

data = {}
while dayObs <= endDay:
    if dayObs > 20250000:
        butler = Butler('/repo/embargo', collections="LATISS/raw/all")
        
    records = butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs)
    #print(dayObs, records.count())
    if records.count() < 200:
        dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                       
        continue
    exposureList = []
    for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
        if (record.observation_type=='science') and (record.exposure_time > 29.0) and (record.physical_filter=='empty~holo4_003'):
            exposureList.append(record.id)
    exposureList = sorted(exposureList)
    print(dayObs, len(exposureList))

    fwhms = []
    for expId in exposureList:
        try:
            exp = butler.get('raw', detector=0, exposure=expId, instrument='LATISS')
            isrResult = isrTask.run(exp) # This runs the ISR
            xs = list(range(x1, x2))
            fluxes = isrResult.exposure.image.array[y, x1:x2]
            args = [xs, fluxes]
            x0 = [100, 8000.0, 1750, 20]
            bounds=[(0, 10000), (1000, 200000), (1700, 1800), (2, 20)]
            result = minimize(FOM, x0, args=args, bounds=bounds, method='Powell')
            [bg, mag, mean, sigma] = result.x
            fwhm = sigma / 10.0 * 2.355
            fwhms.append(fwhm)
            print(expId, fwhm)
        except:
            print(f"{expId} failed")
            continue
    data[dayObs] = fwhms
    print(dayObs, np.median(fwhms))
    dayObs = calcNextDay(dayObs)  # find the next dayObs                                                                                           
    outfile = open('/home/c/cslage/u/AuxTel/fwhm/AuxTel_Spectral_FWHM_21Jan25.pkl', 'wb')
    pkl.dump(data, outfile)
    outfile.close()
    dayObs = calcNextDay(dayObs)  # find the next dayObs



In [ ]:
infile = open('/home/c/cslage/u/AuxTel/fwhm/AuxTel_Spectral_FWHM_21Jan25.pkl', 'rb')
data = pkl.load(infile)
infile.close()
dates = data.keys()
print(dates)


In [ ]:
infile = open('/home/c/cslage/u/AuxTel/fwhm/AuxTel_Spectral_FWHM_30Jan25.pkl', 'rb')
data = pkl.load(infile)
infile.close()
dates = data.keys()
print(dates)


In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,8))
xaxis = []
yaxis = []
xticks = []
xticklabels = []
tot_2024 = []
tot_2025 = []
counter = 0
for dayObs in data.keys():
    fwhms = data[dayObs]
    #print(dayObs, len(fwhms))
    if len(fwhms) < 10:
        continue
    if np.min(fwhms) > 3.0:
        continue
    for fwhm in fwhms:
        xaxis.append(counter)
        yaxis.append(fwhm)
        if dayObs < 20250101:
            tot_2024.append(fwhm)
        else:
            tot_2025.append(fwhm)
    xticks.append(counter)
    xticklabels.append(dayObs)
    counter += 1

ax.set_ylim(0,3.0)
ax.scatter(xaxis, yaxis)
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels, rotation=90)
ax.vlines(30.5, 0, 3.0, ls='--', color='red', lw=4)
ax.set_title("AuxTel spectrum FWHM empty~holo4_003") 
ax.set_ylabel("FWHM (arcseconds)")
ax.text(10, 0.2, f"Median FWHM = {np.median(tot_2024):.2f} arcsec")
ax.text(31, 0.2, f"Median FWHM = \n{np.median(tot_2025):.2f} arcsec")
plt.savefig('/home/c/cslage/u/AuxTel/fwhm/Spectrum_FWHM_30Jan25.png')